In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/DarkPhoenix33/DataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="DarkPhoenix33"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f3ac4c7b2d9b3e6628e12ebe236ab70f27ac31ee"

In [3]:
os.chdir("../")
%pwd


'c:\\D\\MLOps\\DataScienceProject'

In [14]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [15]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/DarkPhoenix33/DataScienceProject.mlflow"
        )
        return model_evaluation_config




In [17]:
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse= np.sqrt(mean_squared_error(actual,pred))
        mae= mean_absolute_error(actual,pred)
        r2= r2_score(actual,pred)
        return rmse, mae, r2
    def log_into_mlflow(self):

        test_data= pd.read_csv(self.config.test_data_path)
        model= joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualtites = model.predict(test_x)

            (rmse, mae, r2)= self.eval_metrics(test_y,predicted_qualtites)

            #Saving metrics as local

            scores={"rmse":rmse, "mae": mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)


            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            #Model registry does not work with file store

            if tracking_url_type_store != "file":

                #Register the model
                #There are other ways to use the model reigstry, which depenmds on the use case,
                # please refer to the doc for more information:
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


        

In [23]:
try:
    config= ConfigurationManager()
    model_evaluation_config= config.get_model_evaluation_config()
    model_evaluation= ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e



[2024-12-20 17:19:32,494:INFO:common: yaml file:config\config.yaml loaded successfully]
[2024-12-20 17:19:32,497:INFO:common: yaml file:params.yaml loaded successfully]
[2024-12-20 17:19:32,501:INFO:common: yaml file:schema.yaml loaded successfully]
[2024-12-20 17:19:32,503:INFO:common: Created dfirectory at: artifacts_root]
[2024-12-20 17:19:32,504:INFO:common: Created dfirectory at: artifacts/model_evaluation]
[2024-12-20 17:19:32,840:INFO:common: json file saved at; artifacts\model_evaluation\metrics.json]


2024/12/20 17:19:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/12/20 17:19:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


üèÉ View run funny-cod-215 at: https://dagshub.com/DarkPhoenix33/DataScienceProject.mlflow/#/experiments/0/runs/d96bc3a422ad479b8f441c6f32801624
üß™ View experiment at: https://dagshub.com/DarkPhoenix33/DataScienceProject.mlflow/#/experiments/0
